In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.2 MB/s

In [2]:
import gradio as gr
import joblib
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
import numpy as np

In [4]:
modelo1 = joblib.load('/content/drive/MyDrive/Proyecto Integrador 2023-2/modelofinal.joblib')

In [5]:
def preprocess_data(filepath):
  columns=['npedido', 'nit', 'nombrecliente', 'ciudad', 'codref', 'productoean',
        'descripcion', 'presentacion', 'cantidad', 'precio unitario',
        'subtotal', 'periodo', 'linea', 'fechapedido', 'canal', 'zona',
        'cantidadfactura', 'preciofactura', 'subtotalfacturado']
  df = pd.read_csv(filepath,on_bad_lines='skip',sep=',',names=columns,nrows=20)
  df['productoean'] = df['productoean'].astype('object')
  categoria_deseada = 'Acetaminofen 150 mg/ 5mL Jarabe'
  df= df[df['descripcion'] == categoria_deseada]
  df =  df.drop(columns = ['npedido', 'nit',  'codref',
    'subtotal', 'periodo', 'subtotalfacturado','descripcion','linea','zona','cantidadfactura', 'preciofactura', 'subtotalfacturado','cantidad'])
  # df['fechapedido'] = pd.to_datetime(df['fechapedido'])
  # df = df.sort_values(by='fechapedido')
  cat_cols = df.select_dtypes(include=object).columns
  #Definimos nuestro Pipeline de pre-procesamiento
  categorical_transformer = Pipeline(
      steps=[("encoder", OrdinalEncoder())]
  )
  preprocessor = ColumnTransformer(
      transformers=[
          ('cat', categorical_transformer, cat_cols)
      ],
      remainder='passthrough'
      )
  # # Fit and transform the DataFrame using the ColumnTransformer
  transformed_df= preprocessor.fit_transform(df)
  transformed_columns = preprocessor.get_feature_names_out()
  # # Elimina los prefijos de las columnas ('cat__' y 'remainder__')
  cleaned_columns = [col.replace('cat__', '').replace('remainder__', '') for col in transformed_columns]
  # # Crea un nuevo DataFrame con los datos transformados y los nombres de las columnas
  encoded_df = pd.DataFrame(transformed_df, columns=cleaned_columns)
  encoded_df=encoded_df.drop('fechapedido', axis=1)
  # Realiza la predicción con el modelo
  preds = modelo1.predict(encoded_df)
# Crear una serie de Pandas con los resultados predichos
  resultados_df = pd.Series(preds, name='Demanda_Predicha')
  # Reiniciar el índice de df para que sea una secuencia numérica
  df = df.reset_index(drop=True)
  # Agregar la serie como una nueva columna al DataFrame df
  df['Demanda_Predicha'] = resultados_df
  return df

In [6]:
demo = gr.Interface(fn=preprocess_data, inputs=gr.File(type="filepath"), outputs=gr.Textbox())
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4dd75f6b3073beba67.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4dd75f6b3073beba67.gradio.live
